# 转换数据



    类别型数据  
类别型数据（Categorical data）是指具有有限个离散取值的数据类型。它表示了某个特征或变量的不同类别或类别级别。类别型数据通常用于描述性别、颜色、地区、产品类型等具有明确分类的特征。  

类别型数据与数值型数据不同，数值型数据是连续的、可度量的数据，而类别型数据则是离散的、不可度量的数据。  
类别型数据可以分为两种类型：名义型数据（Nominal data）和有序型数据（Ordinal data）。   

名义型数据：名义型数据是没有任何顺序或等级关系的类别型数据。例如，颜色（红、绿、蓝）和地区（东、南、西、北）就是名义型数据。对于名义型数据，只能判断是否相等，不能比较大小。\  

有序型数据：有序型数据是具有一定顺序或等级关系的类别型数据。例如，教育程度（小学、初中、高中、大学）和满意度评级（非常不满意、不满意、一般、满意、非常满意）就是有序型数据。对于有序型数据，可以进行比较和排序。  
在数据分析和机器学习中，对于类别型数据的处理通常需要进行哑变量处理、标签编码或者使用特定的模型算法来处理。  

    1哑变量处理见PPT P37
哑变量处理会将每个类别转换为一个新的二进制列，机器学习算法更容易处理，同时保留了原始数据的信息。这样，你就可以在处理类别型数据时更好地应用机器学习算法。






***


In [2]:
import pandas as pd
all_info = pd.read_csv('./data/user_all_info.csv')
live_type = all_info.loc[0: 5, '居住类型']   #抽取部分数据做演示
print('哑变量处理前的数据为：\n', live_type)
print('哑变量处理后的数据为：\n', pd.get_dummies(live_type))

哑变量处理前的数据为：
 0    城市
1    城市
2    农村
3    农村
4    城市
5    城市
Name: 居住类型, dtype: object
哑变量处理后的数据为：
       农村     城市
0  False   True
1  False   True
2   True  False
3   True  False
4  False   True
5  False   True


***

2离散化连续型数据 P41   
https://zhuanlan.zhihu.com/p/63990922
一、概念

某些分类算法，要求我们对连续性的属性进行分类处理，离散化的过程主要包括确定分类的个数，并将数据集映射到这些分类中，这里涉及三种分类方法：

1）等宽法

类似于制作频数分布图，将属性分布值分为几个等分的分布区间；

2）等频法

将相同数量的记录放入每个区间；

3）基于聚类的分析方法

将属性按照K-means算法进行聚类，然后根据聚类的分类，将同一聚类的记录合并到同一组内。

In [4]:
import pandas as pd

# 创建一个包含连续型数据的DataFrame
data = pd.DataFrame({'score': [85, 92, 78, 80, 95, 88, 75, 98, 87]})

# 定义离散化的区间
bins = [0, 60, 70, 80, 90, 100]

# 使用cut()函数进行离散化处理
data['category'] = pd.cut(data['score'], bins)

# 打印结果
print(data)

   score   category
0     85   (80, 90]
1     92  (90, 100]
2     78   (70, 80]
3     80   (70, 80]
4     95  (90, 100]
5     88   (80, 90]
6     75   (70, 80]
7     98  (90, 100]
8     87   (80, 90]


***
2）等宽离散法

In [6]:
# 代码 4-25
age_cut = pd.cut(all_info['年龄'], 5,labels=range(5))
print('离散化后5条记录年龄分布为：\n', age_cut.value_counts())

离散化后5条记录年龄分布为：
 年龄
3    767
2    733
4    661
1     61
0      7
Name: count, dtype: int64


***
(2)等频法离散化

举个例子来说明，假设data是一个包含100个数据点的Series，k的值为5。那么np.arange(0, 1 + 1.0 / k, 1.0 / k)将生成一个等差数列[0, 0.25, 0.5, 0.75, 1]。然后，data.quantile()函数将计算数据data在这些分位数位置上的值，即计算数据的最小值、25%分位数、50%分位数、75%分位数和最大值。
这样做的目的是为了将数据按照等频离散化的要求进行划分，确保每个区间中的数据点数量相等。

In [7]:
# 代码 4-26
import numpy as np
# 自定义等频法离散化函数
def same_rate_cut(data, k):
    w = data.quantile(np.arange(0, 1 + 1.0 / k, 1.0 / k))
    data = pd.cut(data, w)
    return data
# 对用户年龄进行等频法离散化
age_same_rate = same_rate_cut(all_info['年龄'], 5).value_counts()   
print('用户年龄数据等频法离散化后分布状况为：', '\n', age_same_rate)

用户年龄数据等频法离散化后分布状况为： 
 年龄
(-5.0, 21.0]    501
(27.0, 33.0]    472
(21.0, 27.0]    438
(33.0, 39.0]    432
(39.0, 45.0]    385
Name: count, dtype: int64


***


In [ ]:
%conda install scikit-learn

In [16]:
# 代码 4-27
# 自定义数据K-Means聚类离散化函数
def kmean_cut(data, k):
    from sklearn.cluster import KMeans
    # 建立模型
    kmodel = KMeans(n_clusters=k)
    kmodel.fit(data.values.reshape((len(data), 1)))  # 训练模型
    # 输出聚类中心并排序
    c = pd.DataFrame(kmodel.cluster_centers_).sort_values(0)   
    w = c.rolling(2).mean().iloc[1:]  # 相邻两项求中点，作为边界点
    w = [0] + list(w[0]) + [data.max()]  # 把首末边界点加上
    data = pd.cut(data, w)
    return data
# 用户年龄等频法离散化
all_info.dropna(inplace=True)
age_kmeans = kmean_cut(all_info['年龄'], 5).value_counts()   
print('用户年龄聚类离散化后各个类别数目分布状况为：', '\n', age_kmeans)

d:\tool\anaconda3\envs\myenvironment\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


用户年龄聚类离散化后各个类别数目分布状况为： 
 年龄
(20.511, 27.33]     502
(27.33, 33.588]     452
(33.588, 39.501]    424
(0.0, 20.511]       405
(39.501, 45.0]      379
Name: count, dtype: int64
